In [2]:
import numpy as np
import os
import json
import numpy.random

In [ ]:
in_folders = ['art_201811', 'sports']

In [27]:
SMALL_PERCENTILE = 90
LARGE_PERCENTILE = 99

In [14]:
def get_data(in_folder, max_number, number_count, file_pattern, zfill_number):
    file_numbers = numpy.random.choice(np.arange(1,max_number+1), size=number_count,replace=False)

    data_array = []
    for file_number in file_numbers:
        file_name = os.path.join(os.getcwd(),in_folder,file_pattern+'%s.json'%(str(file_number).zfill(zfill_number)))
                                 
        with open(file_name,'r', encoding='utf8') as json_file:  
            data = json.load(json_file)
            p = {}
            p['data'] = data['text']
            p["title"] = data["title"]
            p["site_section"] = data['thread']["site_section"]
            p["published"] = data["published"]
            data_array.append(p)
    return data_array

In [28]:
from nltk.corpus import wordnet

In [18]:
from nltk.corpus import stopwords

# You will have to download the set of stop words the first time
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
# Load stop words
stop_words = stopwords.words('english')
nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('names')
nltk.download('averaged_perceptron_tagger')

english_names = nltk.corpus.names.words()

def remove_names(words):
    words_filtered = []
    
    for w in words:
        if w not in english_names:
            words_filtered.append(w)
    return words_filtered

def remove_stop_words(words):
    words_filtered = []
    
    for w in words:
        if w not in stop_words:
            words_filtered.append(w)
    return words_filtered

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\TL\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\TL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [21]:
def get_nouns(data):
    result = []
    #text = word_tokenize("They refuse to permit us to obtain the refuse permit")
    tagged_words = nltk.pos_tag(data)
    for tagged_word in tagged_words:
        if tagged_word[1] in ['NN','NNS']:
            result.append(tagged_word)
    return result

In [22]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import wikipediaapi

In [24]:
def get_wikipedia(keyword):
    wiki_wiki = wikipediaapi.Wikipedia('en')
    page_py = wiki_wiki.page(keyword)
    if page_py.exists():
        return (page_py.summary, page_py.categories)
    return None

In [25]:
import re
def filter_categories(cats):
    cats_ = cats.copy()
    list_ = ['Wiki','pages needing','accuracy disputes','redlink','category link','Pages containing'
             ,'Articles ','Vague or ambiguous','Good articles','Articles lack','Wikipedia pages'
             ,' symbols','Wikipedia articles','Pages with','CS','All Wikipedia articles'
             ,'Use American','Use British English','disambiguation','Disambiguation','-language sources'
             ,'dmy dates','mdy dates','Webarchive template','All articles','Articles containing','Articles with', 
             'Pages using','link template']
     
    for cat in cats:
        if any(l_ in cat for l_ in list_):
            cats_.pop(cat)
    result = []
    for cat in cats_:
        s1 = cat.replace("Category:","")
        result.append(s1)
    return result

In [42]:
def get_all_categories(data_array):
    set_categories = set()
    all_categories = []
    for data in data_array:
        for cat in data['categories']:
            for c in cat:
                set_categories.add(c)
                all_categories.append(c)
            
    return all_categories

"""
def remove_stop_categries(all_categories):
    cats_ = all_categories.copy()
    list_ = ['Wiki','pages needing','accuracy disputes','redlink','category link','Pages containing'
             ,'Articles ','Vague or ambiguous','Good articles','Articles lack','Wikipedia pages'
             ,' symbols','Wikipedia articles','Pages with','CS','All Wikipedia articles'
             ,'Use American','Use British English','disambiguation','Disambiguation','-language sources'
             ,'dmy dates','mdy dates','Webarchive template','All articles','Articles containing','Articles with' 
             ,'Pages using','link template','time', 'calendar', 'Calendar', 'date', 'month','day of', 'Year', 'Units of'
             ,'Customary units', 'Integer','Punctuation', 'All self-contradictory articles', 'Weeks'
            ]

    for cat in all_categories:
        if any(l_ in cat for l_ in list_):
            cats_.remove(cat)
    return cats_
"""

"\ndef remove_stop_categries(all_categories):\n    cats_ = all_categories.copy()\n    list_ = ['Wiki','pages needing','accuracy disputes','redlink','category link','Pages containing'\n             ,'Articles ','Vague or ambiguous','Good articles','Articles lack','Wikipedia pages'\n             ,' symbols','Wikipedia articles','Pages with','CS','All Wikipedia articles'\n             ,'Use American','Use British English','disambiguation','Disambiguation','-language sources'\n             ,'dmy dates','mdy dates','Webarchive template','All articles','Articles containing','Articles with' \n             ,'Pages using','link template','time', 'calendar', 'Calendar', 'date', 'month','day of', 'Year', 'Units of'\n             ,'Customary units', 'Integer','Punctuation', 'All self-contradictory articles', 'Weeks'\n            ]\n\n    for cat in all_categories:\n        if any(l_ in cat for l_ in list_):\n            cats_.remove(cat)\n    return cats_\n"

In [45]:
def check_create_folder(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [50]:
def add_categories(data_array, in_folder):
    for count in range(3,101):
        data = data_array[count]
        print(count)
        data['categories'] = []
        sentenses = split_into_sentences(data['data'])
        for sentense in sentenses:
            words = word_tokenize(sentense)
            words = remove_names(words)
            words = remove_stop_words(words)
            nouns = get_nouns(words)
            categories = []
            for noun in nouns:
                wiki = get_wikipedia(noun[0])
                if wiki:
                    cats = filter_categories(wiki[1])
                    data['categories'].append(cats)
        check_create_folder(os.path.join(os.getcwd(), in_folder,'json'))
        with open(os.path.join(os.getcwd(), in_folder,'json','data_array%s.json'%(str(count))), 'w') as fout:
            json.dump(data , fout)            

In [37]:
def prune_data_array(data_array):
    max_value = list(data_array['class'][0].values())[0]
    cnt = 0
    items = []
    for item in data_array['class']:
        if max_value > list(item.values())[0]:
            max_value = list(item.values())[0]
            cnt += 1
            if cnt > 2:
                data_array['class'] = items
                return data_array
        items.append(item)    
        #print(item)
    return data_array     

In [40]:
def get_classification_results(data_array, in_folder, keyword):
    allsyns_array = []
    for i in range(0,len(data_array)):
        data_array[i] = prune_data_array(data_array[i])
        word_list = [list(item.keys())[0] for item in data_array[i]['class']]

        allsyns1 = set(ss for word in word_list for ss in wordnet.synsets(word, pos=wordnet.NOUN))
        if len(allsyns1) > 0:
            allsyns2 = set(wordnet.synsets(keyword, pos=wordnet.NOUN))
            best = max((wordnet.wup_similarity(s1, s2) or 0, s1, s2) for s1, s2 in 
                product(allsyns1, allsyns2))
            worst = min((wordnet.wup_similarity(s1, s2) or 0, s1, s2) for s1, s2 in 
                product(allsyns1, allsyns2))
            print((best[0]+worst[0])/2)
            data = {'data':data_array[i]['data'],'best':best[0], 'worst':worst[0]}
            allsyns_array.append(data)
            check_create_folder(os.path.join(os.getcwd(),in_folder,'3level_theme'))
            with open(os.path.join(os.getcwd(),in_folder,'3level_theme','data_array%s.json'%(str(i))), 'w') as fout:
                json.dump(data , fout)   
    return allsyns_array

In [41]:
#data_array_business = get_data('business', 14794, 1000, 'news_00', 5)

data_array_art = get_data('art_201811', 100, 100, 'post_', 3)

len(data_array_art)

100

In [46]:
check_create_folder(os.path.join(os.getcwd(), 'art_201811','json'))

In [ ]:
data_array_art = add_categories(data_array_art, 'art_201811')

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [ ]:
allsyns_array = get_classification_results(data_array_art, 'art_201811', 'science')